In [1]:
import pandas as pd
from pymongo import MongoClient
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from pandas.io.json import json_normalize


In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


In [3]:
df = list(db.companies.find({'$and': 
                    [ {'deadpooled_year':None}, 
                        {'founded_year': {'$gt':2010}},
                        {'offices': { '$ne': [] } },
                        {'$or': [{'category_code':'games_video'},
                                {'category_code':'software'},
                                {'category_code ': 'design'}] 
                    }]}
                    , {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                        "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
                    ).sort('number_of_employees', -1))


In [4]:
data=pd.DataFrame(df)
data['category_code'].value_counts()

games_video    8
software       6
Name: category_code, dtype: int64

In [5]:


data['country']= data['offices'].apply(lambda elem : elem[0]['country_code'])
data['city']= data['offices'].apply(lambda elem : elem[0]['city'])
data['lat']= data['offices'].apply(lambda elem : elem[0]['latitude'])
data['long']= data['offices'].apply(lambda elem : elem[0]['longitude'])


In [6]:
data

,_id,name,category_code,number_of_employees,founded_year,offices,country,city,lat,long
0,52cdef7c4bab8bd67529831a,Social Gaming Network,games_video,100.0,2011,"[{'city': 'Los Angeles', 'country_code': 'USA'...",USA,Los Angeles,37.446823,-122.161523
1,52cdef7d4bab8bd675299dae,Navara,software,80.0,2012,"[{'city': 'Maarssen', 'country_code': 'NLD', '...",NLD,Maarssen,52.126616,5.038078
2,52cdef7e4bab8bd67529a716,Unison Technologies,software,30.0,2011,"[{'city': 'New York', 'country_code': 'USA', '...",USA,New York,40.764577,-73.979901
3,52cdef7c4bab8bd675298523,Simplicant,software,10.0,2012,"[{'city': 'Palo Alto', 'country_code': 'USA', ...",USA,Palo Alto,NaN,NaN
4,52cdef7d4bab8bd675298f55,Pixelmatic,games_video,10.0,2011,"[{'city': 'Vancouver', 'country_code': 'CAN', ...",CAN,Vancouver,49.263588,-123.138565
5,52cdef7d4bab8bd675298a4d,Fuzz,games_video,6.0,2011,"[{'city': 'Santa Clara', 'country_code': 'USA'...",USA,Santa Clara,37.760524,-122.387799
6,52cdef7e4bab8bd67529bcf4,Molecube,games_video,6.0,2011,"[{'city': 'Quebec', 'country_code': 'CAN', 'la...",CAN,Quebec,NaN,NaN
7,52cdef7d4bab8bd675298da5,Carfeine,software,5.0,2012,"[{'city': 'Bethesda', 'country_code': 'USA', '...",USA,Bethesda,38.989124,-77.026676
8,52cdef7e4bab8bd67529b8bb,Hayen Company,games_video,1.0,2011,"[{'city': 'Columbia', 'country_code': 'USA', '...",USA,Columbia,34.019114,-81.004871
9,52cdef7d4bab8bd675298f32,Mousebreaker,games_video,NaN,2011,"[{'city': 'Hilton', 'country_code': 'GBR', 'la...",GBR,Hilton,NaN,NaN


In [7]:

data= data.drop(['_id', 'offices'], axis=1)
data= data.dropna(subset=['lat', 'long'])


In [8]:
data

,name,category_code,number_of_employees,founded_year,country,city,lat,long
0,Social Gaming Network,games_video,100.0,2011,USA,Los Angeles,37.446823,-122.161523
1,Navara,software,80.0,2012,NLD,Maarssen,52.126616,5.038078
2,Unison Technologies,software,30.0,2011,USA,New York,40.764577,-73.979901
4,Pixelmatic,games_video,10.0,2011,CAN,Vancouver,49.263588,-123.138565
5,Fuzz,games_video,6.0,2011,USA,Santa Clara,37.760524,-122.387799
7,Carfeine,software,5.0,2012,USA,Bethesda,38.989124,-77.026676
8,Hayen Company,games_video,1.0,2011,USA,Columbia,34.019114,-81.004871
10,Kidos,games_video,NaN,2011,USA,New York,40.768058,-73.956599
11,Kidos,games_video,NaN,2011,USA,New York,40.768058,-73.956599
12,VisualOn,software,NaN,2011,USA,Santa Clara,37.270518,-121.955879


In [9]:
design = list(db.companies.find({'category_code': 'design'},
                    {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                        "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}))
                    



In [13]:
design= pd.DataFrame(design)
design

,_id,name,category_code,number_of_employees,founded_year,offices
0,52cdef7c4bab8bd675298447,99designs,design,100.0,2008,"[{'city': 'San Francisco', 'country_code': 'US..."
1,52cdef7e4bab8bd67529ba4e,Graticle,design,1.0,2009,"[{'city': 'Ellensburg', 'country_code': 'USA',..."
2,52cdef7e4bab8bd67529bd1a,Kickstarter,design,NaN,2009,"[{'city': 'Brooklyn', 'country_code': 'USA', '..."
3,52cdef7f4bab8bd67529c47a,Moonfruit,design,NaN,2000,"[{'city': 'London', 'country_code': 'GBR', 'la..."


In [17]:
design['country']= design['offices'].apply(lambda elem : elem[0]['country_code'])
design['city']= design['offices'].apply(lambda elem : elem[0]['city'])
design['lat']= design['offices'].apply(lambda elem : elem[0]['latitude'])
design['long']= design['offices'].apply(lambda elem : elem[0]['longitude'])

In [21]:
design= design.drop(['_id', 'offices'], axis=1)
design= design.dropna(subset=['lat', 'long'])

In [22]:
design.dropna()

,name,category_code,number_of_employees,founded_year,country,city,lat,long
0,99designs,design,100.0,2008,USA,San Francisco,37.795531,-122.400598
1,Graticle,design,1.0,2009,USA,Ellensburg,45.796841,-122.693144


In [25]:
result = pd.concat([data, design], ignore_index=True)

In [26]:
result

,name,category_code,number_of_employees,founded_year,country,city,lat,long
0,Social Gaming Network,games_video,100.0,2011,USA,Los Angeles,37.446823,-122.161523
1,Navara,software,80.0,2012,NLD,Maarssen,52.126616,5.038078
2,Unison Technologies,software,30.0,2011,USA,New York,40.764577,-73.979901
3,Pixelmatic,games_video,10.0,2011,CAN,Vancouver,49.263588,-123.138565
4,Fuzz,games_video,6.0,2011,USA,Santa Clara,37.760524,-122.387799
5,Carfeine,software,5.0,2012,USA,Bethesda,38.989124,-77.026676
6,Hayen Company,games_video,1.0,2011,USA,Columbia,34.019114,-81.004871
7,Kidos,games_video,NaN,2011,USA,New York,40.768058,-73.956599
8,Kidos,games_video,NaN,2011,USA,New York,40.768058,-73.956599
9,VisualOn,software,NaN,2011,USA,Santa Clara,37.270518,-121.955879


In [27]:
result.to_json("input/companies_needs.json",orient="records")

## companies of design and gaming in tableau:
https://public.tableau.com/profile/albert8801#!/vizhome/companies_game_design/Hoja1?publish=yes